# Projeto 1 - Ciência dos Dados

Nome: Breno Quessie Barbosa de Lima

Nome: Davi Reis Vieira de Souza

**Atenção:** Serão permitidos grupos de três pessoas, mas com uma rubrica mais exigente. Grupos deste tamanho precisarão fazer um questionário de avaliação de trabalho em equipe

___
Carregando algumas bibliotecas:

In [1]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os

**Em `filename`, coloque o nome do seu arquivo de dados!**

In [2]:
import os

filename = 'skybrasil.xlsx'
if filename in os.listdir():
    print(f'Encontrei o arquivo {filename}, tudo certo para prosseguir com a prova!')
else:
    print(f'Não encontrei o arquivo {filename} aqui no diretório {os.getcwd()}, será que você não baixou o arquivo?')

Encontrei o arquivo skybrasil.xlsx, tudo certo para prosseguir com a prova!


Carregando a base de dados com os tweets classificados como relevantes e não relevantes:

In [3]:
train = pd.read_excel(filename)
train.head(5)

,Classificação,Treinamento
0,2,@skybrasil poxa.. em plena pandemia subindo pr...
1,2,@skybrasil bora arrumar isso ai pelo amor de d...
2,2,@maurinhodutra @skybrasil @sbtonline @libertad...
3,1,@coalhadatricolo @canalesporte09 @skybrasil @s...
4,2,um forte abraço pra @skybrasil chuva. trovejo...


In [4]:
test = pd.read_excel(filename, sheet_name = 'Teste')
test.head(5)

,Classificação,Teste
0,0.0,@maurinhodutra @skybrasil @sbtonline @libertad...
1,1.0,@skyresponde @skybrasil @proconrj @reclameaqui...
2,0.0,rt @minhaoperadora: nova opção chega para subs...
3,2.0,rt @danielgonzaga: a @skyresponde @skybrasil p...
4,0.0,@christian_cruel @skybrasil larga essa sky gat...


In [5]:
train['Classificação'].value_counts()

2    263
1    123
0    114
Name: Classificação, dtype: int64

In [6]:
test['Classificação'].value_counts()

2.0    70
0.0    45
1.0    38
Name: Classificação, dtype: int64

___
## Classificador automático de sentimento


Faça aqui uma descrição do seu produto e o que considerou como relevante ou não relevante na classificação dos tweets.

ESCREVA AQUI...

___
### Montando um Classificador Naive-Bayes

Considerando apenas as mensagens da planilha Treinamento, ensine  seu classificador.

___
### Verificando a performance do Classificador

Agora você deve testar o seu classificador com a base de Testes.

___
### Concluindo

___
## Aperfeiçoamento:

Os trabalhos vão evoluir em conceito dependendo da quantidade de itens avançados:

* Limpar: \n, :, ", ', (, ), etc SEM remover emojis
* Corrigir separação de espaços entre palavras e emojis ou entre emojis e emojis
* Propor outras limpezas e transformações que não afetem a qualidade da informação ou classificação
* Criar categorias intermediárias de relevância baseadas na probabilidade: ex.: muito relevante, relevante, neutro, irrelevante, muito irrelevante (3 categorias: C, mais categorias conta para B)
* Explicar por que não posso usar o próprio classificador para gerar mais amostras de treinamento
* Propor diferentes cenários para Naïve Bayes fora do contexto do projeto
* Sugerir e explicar melhorias reais com indicações concretas de como implementar (indicar como fazer e indicar material de pesquisa)
* Montar um dashboard que realiza análise de sentimento e visualiza estes dados

___
## Referências

[Naive Bayes and Text Classification](https://arxiv.org/pdf/1410.5329.pdf)  **Mais completo**

[A practical explanation of a Naive Bayes Classifier](https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/) **Mais simples**